In [2]:
# necessary for DenodoConnection
#%pip install psycopg2-binary
#%pip install shap

#%pip install scikit-optimize
# generate train and validation sets
# during training, make K models via k-folds cross validation
# during evaluation, evaluate on validation set (completely left out).

# stages 1, 3 and 4 for Corn Brazil Summer
# do we combine stages to make 1 model?

In [1]:
# import packages
import os
import sys

import pandas as pd, numpy as np
import argparse
import xgboost

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.performance_lib import predictive_advancement_lib
from libs.performance_lib import preprocessor_class
from libs.performance_lib import performance_helper
from sklearn.metrics import balanced_accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ap_data_sector = 'CORN_BRAZIL_SAFRINHA'
bucket = 'us.com.syngenta.ap.nonprod'

In [3]:
# when testing, we read directly from S3, bucket is provided as a separate input to function, don't include in input args. 
input_args = [
    '--s3_input_training_data_folder', 'uat/dme/performance/reformatting_performance_pipeline_temp_out/data'
]

parser = argparse.ArgumentParser(description='app inputs and outputs')
parser.add_argument('--s3_input_training_data_folder', type=str,
                    help='s3_input_training_data_folder', required=True)
input_args_parsed = parser.parse_args(input_args)

In [4]:
df_tr_all = pd.read_csv('s3://' + os.path.join(bucket, input_args_parsed.s3_input_training_data_folder, ap_data_sector, 'adv_model_training_data.csv'))

In [14]:
trait_list = ['ERHTN','GMSTP','HAVPN','LRTLR','PLHTN','STD_N','STKLR','TWSMN','YGSMN']
df_temp = df_tr_all[df_tr_all['current_stage'] == 4]
df_temp[['was_adv'] + ['result_' + trait for trait in trait_list]].corr()

,was_adv,result_ERHTN,result_GMSTP,result_HAVPN,result_LRTLR,result_PLHTN,result_STD_N,result_STKLR,result_TWSMN,result_YGSMN
was_adv,1.000000,-0.048133,-0.004371,0.178498,-0.031250,-0.049415,0.127520,-0.062373,0.034729,0.062013
result_ERHTN,-0.048133,1.000000,-0.153399,-0.846499,-0.007493,0.773370,-0.449668,0.009121,-0.224243,0.267412
result_GMSTP,-0.004371,-0.153399,1.000000,-0.594088,0.141817,-0.206170,-0.561858,-0.045550,-0.155200,0.340170
result_HAVPN,0.178498,-0.846499,-0.594088,1.000000,0.257984,-0.872253,0.978922,-0.300200,NaN,-0.731718
result_LRTLR,-0.031250,-0.007493,0.141817,0.257984,1.000000,0.034123,-0.156094,0.180699,0.037483,-0.026193
result_PLHTN,-0.049415,0.773370,-0.206170,-0.872253,0.034123,1.000000,-0.365156,-0.071624,-0.255023,0.205020
result_STD_N,0.127520,-0.449668,-0.561858,0.978922,-0.156094,-0.365156,1.000000,-0.154350,-0.848393,-0.400959
result_STKLR,-0.062373,0.009121,-0.045550,-0.300200,0.180699,-0.071624,-0.154350,1.000000,0.202267,-0.131583
result_TWSMN,0.034729,-0.224243,-0.155200,NaN,0.037483,-0.255023,-0.848393,0.202267,1.000000,-0.106003
result_YGSMN,0.062013,0.267412,0.340170,-0.731718,-0.026193,0.205020,-0.400959,-0.131583,-0.106003,1.000000


In [15]:
trait_list = ['ERHTN','GMSTP','HAVPN','LRTLR','PLHTN','STD_N','STKLR','TWSMN','YGSMN']
df_temp = df_tr_all[df_tr_all['current_stage'] == 4]
df_temp[['was_adv'] + ['result_diff_' + trait for trait in trait_list]].corr()

,was_adv,result_diff_ERHTN,result_diff_GMSTP,result_diff_HAVPN,result_diff_LRTLR,result_diff_PLHTN,result_diff_STD_N,result_diff_STKLR,result_diff_TWSMN,result_diff_YGSMN
was_adv,1.000000,0.007940,0.047283,NaN,-0.018319,0.022184,0.006262,-0.048490,-0.099542,0.181465
result_diff_ERHTN,0.007940,1.000000,0.157245,NaN,0.063358,0.617506,-0.005450,-0.116561,-0.044713,0.111927
result_diff_GMSTP,0.047283,0.157245,1.000000,NaN,-0.062391,0.199555,-0.015993,-0.279002,-0.132519,0.026666
result_diff_HAVPN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
result_diff_LRTLR,-0.018319,0.063358,-0.062391,NaN,1.000000,0.042962,-0.003539,0.155370,-0.014493,-0.100126
result_diff_PLHTN,0.022184,0.617506,0.199555,NaN,0.042962,1.000000,0.003886,-0.298413,-0.004225,0.060165
result_diff_STD_N,0.006262,-0.005450,-0.015993,NaN,-0.003539,0.003886,1.000000,0.003787,0.176767,0.019970
result_diff_STKLR,-0.048490,-0.116561,-0.279002,NaN,0.155370,-0.298413,0.003787,1.000000,-0.090331,-0.092762
result_diff_TWSMN,-0.099542,-0.044713,-0.132519,NaN,-0.014493,-0.004225,0.176767,-0.090331,1.000000,-0.084254
result_diff_YGSMN,0.181465,0.111927,0.026666,NaN,-0.100126,0.060165,0.019970,-0.092762,-0.084254,1.000000


In [17]:
cols_to_norm = []
mdl_out_col = 'was_adv'
mdl_year_col = 'analysis_year'
out_year = 2023

df_tr_all[mdl_out_col] = df_tr_all[mdl_out_col].astype(bool)

In [20]:
do_hyperparam_tuning = False    
grouping_cols = ['ap_data_sector','current_stage']
df_tr_all_grouped = df_tr_all.groupby(by=grouping_cols)

# scale pos weight is computed at each fold.
default_xgb_params = {
    'max_depth':5,
    'learning_rate':1e-2,
    'verbosity':0,
    'booster':'gbtree',
    'gamma':0,
    'subsample':0.75,
    'reg_lambda':10,
    'enable_categorical':False,
    'n_estimators':200,
    'eval_metric':balanced_accuracy_score
}

for index, df_use in df_tr_all_grouped:  # build model per unique combination of grouping_cols; stage currently
    df_tr_pred_list = []
    df_te_pred_list = []
    # extract inputs from args
    # get input and output column names

    corr_traits = []
    preproc_steps = []
    corr_score_col = []
    extra_traits = []
    text_traits = []
    if index[1] == 1:
        numeric_raw_traits = ['TWSMN','PLHTN','LRTLR','ERHTN'] 
        numeric_diff_traits = ['YGSMN','STKLR','STD_N','GMSTP'] #'YGSMN-PLHTN-ERHTN-GMSTP', plhtn and erhtn are sparse
        text_traits = [] #'pltqt','notet' at some point?
        
    elif index[1] == 3:
        numeric_raw_traits = ['TWSMN'] 
        numeric_diff_traits = ['YGSMN','STKLR','STD_N','GMSTP','PLHTN','ERHTN','LRTLR'] #'YGSMN-PLHTN-ERHTN-GMSTP', plhtn and erhtn are sparse
        
        #preproc_steps = ['subtract corr']
        #corr_traits = [[['PLHTN','ERHTN','GMSTP'],['YGSMN']]]
        #corr_score_col = 'result_diff' # no '_' after
        
    elif index[1] == 4:
        numeric_raw_traits = ['TWSMN','ERHTN','PLHTN','LRTLR','STKLR','STD_N'] 
        numeric_diff_traits = ['YGSMN'] #'YGSMN-PLHTN-ERHTN-GMSTP', plhtn and erhtn are sparse
        
    mdl_in_cols = extra_traits
    mdl_in_cols.extend(['result_diff_' + trait for trait in numeric_diff_traits])
    mdl_in_cols.extend(['result_' + trait for trait in numeric_raw_traits])

    preproc_class = preprocessor_class.PredAdvPreprocessor(
        preprocess_steps=preproc_steps,
        corr_traits=corr_traits,
        corr_score_col=corr_score_col
        
    )
    
    if do_hyperparam_tuning:
        x = df_use[mdl_in_cols].values
        y = df_use[mdl_out_col].values
        cv_fold_label = df_use['kfold_label'].values

        opt = predictive_advancement_lib.perform_hyperparameter_tuning(
            x=x,
            y=y,
            cv_fold_label=cv_fold_label,
            year_vals=df_use['analysis_year'].values,
            n_iter=20,
            n_points=2,
            n_jobs=2,
            verbose=1
        )
        xgb_params_use = opt.best_estimator_.get_params().copy()
    else:
        opt = []
        xgb_params_use = default_xgb_params.copy()
        
    mdl_list, preproc_class, df_tr_pred, df_te_pred = predictive_advancement_lib.train_models_kfolds(
        df_in=df_use, 
        mdl_in_cols=mdl_in_cols, 
        mdl_out_col=mdl_out_col,
        xgb_params=xgb_params_use,
        preproc_class=preproc_class
    )
    
    acc, conf_mat, roc_auc, f1 = predictive_advancement_lib.get_evaluation_metrics(y_true=df_te_pred[mdl_out_col], y_pred=df_te_pred['recommendation_score']>0.5, y_proba=df_te_pred['recommendation_score'])
    
    print(
        "Train:",
        index,
        df_tr_pred.shape,
        predictive_advancement_lib.get_evaluation_metrics(y_true=df_tr_pred[mdl_out_col], y_pred=df_tr_pred['recommendation_score']>0.5, y_proba=df_tr_pred['recommendation_score'])
    )
    
    print(
        "Test:",
        index,
        df_te_pred.shape,
        predictive_advancement_lib.get_evaluation_metrics(y_true=df_te_pred[mdl_out_col], y_pred=df_te_pred['recommendation_score']>0.5, y_proba=df_te_pred['recommendation_score'])
    )    

    # save 3 files:
    file_id = performance_helper.generate_datetime_id()
    postfix = ap_data_sector+'-year'+str(out_year)+'-stg'+str(int(index[1]))+'-'+ file_id

    mdl_preproc_fname = 'mdl_preprocessor-' + postfix + '.pkl'
    training_data_fname = 'training_data-' + postfix + '.csv'
    meta_info_fname = 'mdl_meta_info-' + postfix + '.pkl'

    # mdl, preproc class together
    mdl_preproc_out = {
        'mdl_list':mdl_list,
        'preprocessor':preproc_class
    }
    
    #meta info
    cv_metrics = {
        'mean_acc' : acc,
        'mean_roc_auc' : roc_auc,
        'mean_f1' : f1,
        'sum_conf_mat' : conf_mat
    }
    meta_out = {
        'opt':opt,
        'mdl_in_cols':mdl_in_cols,
        'mdl_out_cols':mdl_out_col,
        'did_hyper_tune':do_hyperparam_tuning,
        'xgb_params':xgb_params_use,
        'xgb_version':xgboost.__version__,
        'cv_metrics':cv_metrics
    }
    

    # training data as a csv
    local_fpath = '/root/dme_sagemaker/dme_sagemaker/performance_pipeline/preprocess_train_recipes'
    s3_path = os.path.join('uat/dme/performance/reformatting_performance_pipeline_temp_out/data',ap_data_sector)

    performance_helper.write_to_s3(df_te_pred, 
                training_data_fname, 
                local_fpath,
                s3_path=s3_path, 
                bucket=bucket
              )

    performance_helper.write_to_s3(mdl_preproc_out, 
                mdl_preproc_fname, 
                local_fpath,
                s3_path=s3_path, 
                bucket=bucket
              )

    performance_helper.write_to_s3(meta_out, 
                meta_info_fname, 
                local_fpath,
                s3_path=s3_path, 
                bucket=bucket
              )

/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tr_proc['recommendation_score'] = mdl.predict_proba(df_tr_proc[mdl_in_cols])[:,1]
/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:410: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_te_proc['recommendation_score'] = mdl.predict_proba(df_te_proc[mdl_in_cols])[:,1]


Train: ('CORN_BRAZIL_SAFRINHA', 1.0) (149140, 103) (0.8758422449110842, array([[0.82176146, 0.17823854],
       [0.07007697, 0.92992303]]), 0.9351442444645957, 0.3092305231163014)
Test: ('CORN_BRAZIL_SAFRINHA', 1.0) (37285, 103) (0.8541928702805621, array([[0.82127862, 0.17872138],
       [0.11289288, 0.88710712]]), 0.9223913607010408, 0.29655837889996783)


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tr_proc['recommendation_score'] = mdl.predict_proba(df_tr_proc[mdl_in_cols])[:,1]
/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:410: Se

Train: ('CORN_BRAZIL_SAFRINHA', 3.0) (83564, 103) (0.8225037273417228, array([[0.76663667, 0.23336333],
       [0.12162921, 0.87837079]]), 0.8963600801701488, 0.24663800922822102)
Test: ('CORN_BRAZIL_SAFRINHA', 3.0) (20891, 103) (0.7552882468236138, array([[0.76226189, 0.23773811],
       [0.25168539, 0.74831461]]), 0.8287639550606739, 0.21106005387418794)


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tr_proc['recommendation_score'] = mdl.predict_proba(df_tr_proc[mdl_in_cols])[:,1]
/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:410: Se

Train: ('CORN_BRAZIL_SAFRINHA', 4.0) (36172, 103) (0.8447687072687073, array([[0.83174521, 0.16825479],
       [0.14220779, 0.85779221]]), 0.9156127843627844, 0.304097605893186)
Test: ('CORN_BRAZIL_SAFRINHA', 4.0) (9043, 103) (0.7633017133017133, array([[0.82270732, 0.17729268],
       [0.2961039 , 0.7038961 ]]), 0.8237556737556737, 0.24737562756732087)


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [ ]:
#### end training.py

# code below is extra, may be useful elsewhere

In [ ]:
"""
vars_to_plot = ['max_depth','n_estimators']
dims = []
for var in vars_to_plot:
    dims.append(np.argwhere(np.array(list(opt.best_params_.keys())) == var)[0][0])
                            
opt_res = opt.optimizer_results_[0]

x_vals = np.array(opt_res.x_iters)[:,dims[0]]
y_vals = np.array(opt_res.x_iters)[:,dims[1]]
f_vals = opt_res.func_vals

cmap_fun = plt.get_cmap('inferno')

color_vals = cmap_fun(0.75*(1-(f_vals-np.min(f_vals))/(np.max(f_vals)-np.min(f_vals))))

# hotter is better
plt.figure()
#plt.scatter(x_vals,y_vals,color=color_vals)
for i in range(x_vals.shape[0]):
    plt.plot(x_vals[i],y_vals[i],color=color_vals[i],marker='o',markersize=8)
plt.yscale('log')
plt.xlabel(vars_to_plot[0])
plt.ylabel(vars_to_plot[1])
###plt.colorbar()?

df_input_stats = performance_validation_lib.compute_input_statistics(
            df_in=df_tr_proc,
            in_cols=mdl_class.in_cols
        )

#shap code for new class method
# get and plot shapley values after training a model
do_shap = 1
if do_shap == 1:
    import shap
    x = df_te_proc[mdl_class.in_cols].values.astype(float)
    shap_input = shap.utils.sample(x, np.minimum(x.shape[0],1000))
    explainer = shap.Explainer(mdl_class.mdl.predict_proba, shap_input)
    shap_values = explainer(shap_input)
    shap_values = shap_values[:,:,1]

    shap.summary_plot(shap_values, feature_names=mdl_class.in_cols,\
                     plot_type='bar')

    shap.summary_plot(shap_values, feature_names=mdl_class.in_cols,\
                     plot_type='dot')

#for idx in range(len(mdl_class.in_cols)):
#    notnan_mask = np.isnan(shap_input[:,idx])==False
#    if np.sum(notnan_mask) > 10:
#        shap.plots.partial_dependence(idx,mdl_class.mdl.predict,shap_input[notnan_mask,:],ice=False,feature_names=mdl_class.in_cols)

# some useful shapley/interpolation related code
#plt.plot(shap_values[:,0].data, shap_values[:,0].values,'.')
#shap.plots.scatter(shap_values[:,5])

# make sure advancement columns are boolean
df_input = df_te_proc.copy()
df_input['recommendation_score'] = mdl_class.predict_proba(df_input)
if 'was_adv' in df_input.columns:
    df_input['was_adv'] = df_input['was_adv'].astype(bool)
if 'was_adv_next' in df_input.columns:
    df_input['was_adv_next'] = df_input['was_adv_next'].astype(bool)

# set trait column and yield column based on data sector
trait_prefix = 'result_' # this gets both result_ and result_diff cols
yield_col = 'result_diff_YGSMN'

# compute metrics for each stage
adv_metrics_all = []
trait_metrics_all = []

for stg in pd.unique(df_input['current_stage']):
    df_use = df_input[df_input['current_stage'] == stg]
    df_use['random_nums'] = np.random.rand(df_use.shape[0], 1)
    adv_metrics, trait_metrics = performance_validation_lib.compute_model_metrics(
        df_use,
        compute_advancement_metrics=True,
        yield_col=yield_col,
        trait_prefix=trait_prefix
    )
    adv_metrics['stage'] = stg
    trait_metrics['stage'] = stg

    adv_metrics_all.append(adv_metrics)
    trait_metrics_all.append(trait_metrics)

# join across stages
df_adv_metrics = pd.concat(adv_metrics_all,axis=0)
df_trait_metrics = pd.concat(trait_metrics_all,axis=0)
"""